# install and import libraries

In [ ]:
!pip uninstall -y protobuf tokenizers crewai agentops

In [ ]:
!pip install "crewai[agentops]" agentops --upgrade

In [1]:
import os

In [2]:
from pydantic import BaseModel, Field
from typing import List

In [3]:
import agentops
from agentops import track_agent, Session

In [4]:
from crewai import Crew, Agent, Task, Process ,LLM

#Model setup

In [ ]:
from google.colab import userdata
userdata.get('gimini_api')

In [38]:
os.environ["GEMINI_API_KEY"] = userdata.get('gimini_api')

In [ ]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel('gemini-1.5-flash')
response = model.generate_content("what is LLMs")
print(response.text)

In [7]:
import google.generativeai as genai
import os

genai.configure(api_key=os.environ["GEMINI_API_KEY"])
model = genai.GenerativeModel('gemini-1.5-flash')

In [8]:
output_dir = "./ai-agent-output"
os.makedirs(output_dir, exist_ok=True)

basic_llm = LLM(
    # Corrected the model name from "gemini-2.0-flas" to "gemini-1.5-flash"
    model="gemini/gemini-1.5-flash",
    temperature=0,
    provider="google_ai_studio",
    api_key=os.environ["GEMINI_API_KEY"]
)

#Agents

##Agent1 search agent

In [9]:
the_max_queries = 20

In [10]:
class search_recommendation(BaseModel):
    search_queries: List[str] = Field(..., title = "Recommende searches to be send to the search engines" ,  min_items=1, max_items=the_max_queries)

In [11]:
search_recommendation_agent = Agent(
    role="search_recommendation_agent",
    goal="""to provide a list of recommendations search queries to be passed to the search engine.
    The queries must be varied and looking for specific items""",
    backstory="The agent is designed to help in looking for products by providing a list of suggested search queries to be passed to the search engine based on the context provided.",
    llm=basic_llm,
    verbose=True,
)

In [12]:
search_recommendation_task = Task(
    description = "\n".join([
        "Mariam is looking for a job as {job_name}",
        "so the job must be suitable for {level}",
        "The search query must take the best offers",
        "I need links of the jobs",
        "The recommended query must not be more than {the_max_queries} ",
        "The jop must be in {country_name}"
    ]),
    expected_output="A JSON object containing a list of suggested search queries.",
    output_json= search_recommendation,
    agent = search_recommendation_agent,
    output_file=os.path.join(output_dir, "step_1_Recommend _search_queries.json"),

)

##Agent2 google search agent

In [13]:
!pip install -qU tavily-python scrapegraph-py

In [14]:
from tavily import TavilyClient

In [15]:
from crewai.tools import tool

In [40]:
search_clint = TavilyClient(
    api_key= userdata.get('clint_api')
)

In [41]:
class SignleSearchResult(BaseModel):
    title: str
    url: str = Field(..., title="the page url")
    content: str
    score: float
    search_query: str

class AllSearchResults(BaseModel):
    results: List[SignleSearchResult]

In [42]:
@tool
def search_engin_tool(query : str):
  """Useful for search-based queries. Use this to find current information about any query related pages using a search engine"""
  return search_clint.search(query)

In [43]:
search_engine_agent = Agent(
    role="search engine agent",
    goal="To search on job baased on suggested search queries",
    backstory = "that egint desingned to help in finding jobs by using the suggested search queries",
    llm=basic_llm,
    verbose=True,
    tools=[search_engin_tool]
)



In [44]:
search_engine_task = Task(
    description = "\n".join([
        "search for jobs based on the suggested search queries",
        "you have to collect results from the suggested search queries",
        "ignore any results that are not related to the job",
        "Ignore any search results with confidence score less than ({score_th}) ",
        "the search result will be used to summaries the posts to understand what the candidate need to have"
        "you should give me more that 10 jop"

    ]
),
    expected_output="A JSON object containing a search results.",
    output_json= AllSearchResults,
    agent = search_engine_agent,
    output_file=os.path.join(output_dir, "step_2_search_results.json")
)

##Agent3 Scrap agent

In [21]:
from scrapegraph_py import Client

In [45]:
scrape_client = Client(api_key=userdata.get('scrap_api')
)

In [23]:
import json


In [46]:
class ProductSpec(BaseModel):
    specification_name: str
    specification_value: str

class SingleExtractedProduct(BaseModel):
    page_url: str = Field(..., title="The original url of the job page")
    Job_Requirements: str = Field(..., title="The requirements of the job")
    Job_Title: str = Field(..., title="The title of the job")
    Job_Details: str = Field(title="The Details of the job" , default= None)
    Job_Description: str = Field(...,title="The Description of the job" )
    Job_Location: str = Field(title="The location of the job" , default= None)
    Job_Salary: str = Field(itle="The salary of the job" , default= None)
    Job_responsability: str = Field(..., title="The responsibilty of the job")
    Job_type: str = Field(title="The type of the job", default= None)
    Job_Overview : str = Field(..., title="The overview of the job")
    qualifications : str = Field(..., title="The qualifications of the job" )
    product_specs: List[ProductSpec] = Field(..., title="The specifications of the product. Focus on the most important requirements.", min_items=1, max_items=5)
    agent_recommendation_notes: List[str]  = Field(..., title="A set of notes why would you recommend or not recommend this job to the candidate, compared to other jobs.")


class AllExtractedProducts(BaseModel):
    products: List[SingleExtractedProduct]

In [47]:
@tool
def web_scraping_tool(page_url: str):
    """
    An AI Tool to help an agent to scrape a web page
    """
    details = scrape_client.smartscraper(
        website_url=page_url,
        user_prompt="Extract ```json\n" + SingleExtractedProduct.schema_json() + "```\n From the web page"
    )

    return {
        "page_url": page_url,
        "details": details
    }

In [48]:
search_scrap_agent = Agent(
    role="Web scrap agent to extract url information",
    goal = "to extract information from any website",
    backstory= "the egint designed to extract required information from any websie and that information will used to understand which skills the jobs need",
    llm=basic_llm,
    verbose=True,
    tools=[web_scraping_tool]
)

search_scrap_task = Task(
    description = "\n".join([
        "The task is to extract job details from any job offer page url.",
        "The task has to collect results from multiple pages urls.",
        "you should focus on what requirements or qualification or responsibilities",
        "the results from you the user wil use it to understand which skills he need to have"
        "I need you to give me more than +5 jobs"
    ]
),
    expected_output="A JSON object containing jobs details",
    output_json=AllExtractedProducts,
    output_file=os.path.join(output_dir, "step_3_search_results.json"),
    agent=search_scrap_agent
)

##Agent4 summurize agent

In [29]:
search_summirize_agent = Agent(
    role="extract information about what requirments for every job",
    goal = "to extract information about what requirments for every job",
    backstory = "the egint should detecte what requirements for the job according to the job describtion and requirments",
    llm=basic_llm,
    verbose=True,
)

In [33]:
search_summirize_task = Task(
     description = "\n".join([
        "extract what skills shoud the candidate of that job should have",
        "you have to collect results about what each job skills need",
        "ignore any results that have None values",
        "Ignore any search results with confidence score less than ({score_th}) ",
        "the candidate need to understand what skills he should have",
        "you can also recommend skills from understanding jobs title even if it not in the job description"
        "I need you to give me +10 skills"

    ]
),
    expected_output=" Sumurize of what skills that job need candidate to have",
    agent = search_summirize_agent,
    output_file=os.path.join(output_dir, "step_4_search_results.json")
)

#AI crew

In [34]:
Company_Crew = Crew(
    process = Process.sequential,
    agents = [search_recommendation_agent, search_engine_agent , search_scrap_agent,search_summirize_agent],
    tasks = [search_recommendation_task, search_engine_task,search_scrap_task, search_summirize_task]
)

In [35]:
crew_results = Company_Crew.kickoff(
    inputs = { "job_name" : "job offer for AI developer",
              "the_max_queries" : the_max_queries,
               "level" : "senior",
               "score_th" : 0.02,
               "country_name" : "Egypt
    }
)

[EventBus Error] Handler 'on_crew_kickoff_started' failed for event 'CrewKickoffStartedEvent': Could not initialize AgentOps client - API Key is missing.
	    Find your API key at https://app.agentops.ai/settings/projects
# Agent: search_recommendation_agent
## Task: Mariam is looking for a job as job offer for AI developer
so the job must be suitable for senior
The search query must take the best offers
I need links of the jobs
The recommended query must not be more than 20 
The jop must be in Egypt


# Agent: search_recommendation_agent
## Final Answer: 
{
  "search_queries": [
    "AI Developer Senior Jobs Egypt",
    "Senior AI Engineer Jobs Egypt",
    "Senior Machine Learning Engineer Jobs Egypt",
    "Senior AI Scientist Jobs Egypt",
    "Lead AI Engineer Jobs Egypt",
    "Principal AI Engineer Jobs Egypt",
    "AI Architect Jobs Egypt",
    "Senior Data Scientist AI Focus Egypt",
    "Top AI Developer Jobs Egypt",
    "Best Paying AI Jobs Egypt Senior",
    "Senior AI Developer

<ipython-input-25-7ec339fb68ef>:8: PydanticDeprecatedSince20: The `schema_json` method is deprecated; use `model_json_schema` and json.dumps instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  user_prompt="Extract ```json\n" + SingleExtractedProduct.schema_json() + "```\n From the web page"


 

I encountered an error while trying to use the tool. This was the error: [402] Insufficient credits.
 Tool web_scraping_tool accepts these inputs: Tool Name: web_scraping_tool
Tool Arguments: {'page_url': {'description': None, 'type': 'str'}}
Tool Description: 
    An AI Tool to help an agent to scrape a web page
    



# Agent: Wep scrap agent
## Thought: Thought:I need to use the web_scraping_tool to extract the required information from each job URL provided in the context.  The information needed includes Job Requirements, Job Title, Job Details, Job Description, Job Location, Job Salary, Job Responsibility, Job Type, Job Overview, Qualifications, and any other relevant specifications.  I will then organize this information into the specified JSON format.
## Using tool: web_scraping_tool
## Tool Input: 
"{\"page_url\": \"https://aijobs.net/job/1263593-senior-ai-developer/\"}"
## Tool Output: 

I encountered an error while trying to use the tool. This was the error: [402] Insuff